In [28]:
import pandas as pd
import time
import scipy.stats as stats
from scipy import ndimage as ndi
import numpy as np
import pathlib as plb 
import csv
import numpy as np
from skimage import feature
from skimage import exposure
from skimage.io import imread, imsave
from skimage.util import crop
from skimage import data
from skimage.filters import threshold_otsu
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops, regionprops_table
from skimage.morphology import closing, square, remove_small_objects



    
def slots_wells(row):
    row['Slot'] = row['WellNo'][0]
    row['Well'] = row['WellNo'][1]
    return row


def get_dims(flpath, rslt_path, fname):
    label_begin = time.time()
    image = imread(flpath)
    image_nvrt = np.invert(image)
    
    # apply threshold
    print('At threshold')
    thresh = threshold_otsu(image_nvrt)
    print('Threshold: ' + str(thresh))
    bw = closing(image_nvrt > thresh, square(3))

    # remove artifacts connected to image border
    cleared = clear_border(bw)
    #cleared = remove_small_objects(clear_border(bw))
    print('Clearing small objects took ', str(int(time.time() - label_begin)), 'seconds.')

    # label image regions
    label_image = label(bw)
    print('Feature finding and labeling took ', str(int(time.time() - label_begin)), 'seconds.')
    #image_label_overlay = label2rgb(label_image, image=image, bg_label=0)

    props = regionprops_table(label_image, properties=('label','centroid', 'bbox', 'area'))
    dff=pd.DataFrame(props)

    df_area = dff.sort_values(by=['area'], ascending=False)
    image_center = (int(label_image.shape[1]/2),int(label_image.shape[0]/2))


    wells = df_area[(df_area.area>= 2000000) & (df_area.area<=2500000)]
    wells=wells.sort_values(by=['bbox-1'])
    print('Number of wells: ' + str(len(wells)))

    wells.reset_index(drop=True, inplace=True)

    # Sort the plates into the left and right
    mask1 = wells['bbox-1'] > image_center[0]
    df_r = wells[mask1]
    df_l = wells[~mask1]

    ## Split the right plates to upper (#1) and lower (#4)
    dffr=df_r.sort_values(by=['bbox-0'])
    dffr.reset_index(drop=True, inplace=True)


    mask2 = df_r['bbox-0'] > image_center[1]
    dff4 = df_r[mask2]
    dff1 = df_r[~mask2]

    ### Sort the wells on each plate
    df1=dff1.sort_values(by=['bbox-0'])
    df4=dff4.sort_values(by=['bbox-0'])
    df4.reset_index(drop=True, inplace=True)
    df1.reset_index(drop=True, inplace=True)

    ## Split the left plates to upper (#2) and lower (#3)
    dffl=df_l.sort_values(by=['bbox-0'])
    dffl.reset_index(drop=True, inplace=True)

    #MinCol2=1.05*list(dffl.items())[3][1][3]
    mask = df_l['bbox-0'] > image_center[1]
    dff3 = df_l[mask]
    dff2 = df_l[~mask]

    ### Sort the wells on each plate
    df2=dff2.sort_values(by=['bbox-0'])
    df3=dff3.sort_values(by=['bbox-0'])
    df3.reset_index(drop=True, inplace=True)
    df2.reset_index(drop=True, inplace=True)

    ## Update the label of each well
    new_label_1 = pd.Series(['1A', '1B','1C','1D'], name='label', index=[0,1,2,3])
    df1.update(new_label_1)

    new_label_2 = pd.Series(['2A', '2B','2C','2D'], name='label', index=[0,1,2,3])
    df2.update(new_label_2)

    new_label_3 = pd.Series(['3A', '3B','3C','3D'], name='label', index=[0,1,2,3])
    df3.update(new_label_3)

    new_label_4 = pd.Series(['4A', '4B','4C','4D'], name='label', index=[0,1,2,3])
    df4.update(new_label_4)


    ### Append the dataframes
    df_f=df1.append(df2, ignore_index=True).append(df3, ignore_index=True).append(df4, ignore_index=True)

    df_f = df_f.rename(columns={'label': 'WellNo'})
    df_f.apply(lambda row: slots_wells(row), axis=1)

    results_df = cropWell(df_f, image, flpath, rslt_path, fname)
    return results_df
    
        #save_worm_locations(df_f,filtered_worm,path_rslt,label)


In [36]:
### loopwell() is called after the image has been cropped
### This is when the worms are identified, counted and when the CI is calculated
def cropWell(df_f,image, im_path, path_rslt, fname):

    for index, row in df_f.iterrows():

#     fin_image = image[ df_f[][]:Lower_boundary , Left_boundary:Right_boundary ]
        
        fin_image = image[ df_f['bbox-0'][index]:df_f['bbox-2'][index], df_f['bbox-1'][index]:df_f['bbox-3'][index]]

        wellno = df_f['WellNo'][index]
        image_dims = fin_image.shape
        imsave((path_rslt + fname + '_' + wellno + '.tiff'), fin_image)

    return df_f


In [37]:
### This function is called when the user clicks the "Submit" button in the Batch Process window
def batch_process(image_fpath, rslt_path):
    image_folder = plb.Path(image_fpath)
    results_folder = plb.Path(rslt_path)
    results_df = pd.DataFrame()
    for i in image_folder.glob('[!._]*.tif*'):
        pattern = "^[a-zA-Z]"
        fname = i.stem
        image_data = get_dims(i, rslt_path, fname)

        #image_data.to_csv(path_or_buf= results_folder.joinpath(fname + '.csv'))
        results_df = results_df.append(image_data)
        #results_df.head()
    return results_df

In [38]:
img_p = '/Volumes/LaCie/_2021_08_screen/practice_run/Images/'
r_p = '/Volumes/LaCie/_2021_08_screen/practice_run/crops/'
test = batch_process(img_p, r_p)
test.head()

At threshold
Threshold: 124
Clearing small objects took  17 seconds.
Feature finding and labeling took  22 seconds.
Number of wells: 16
At threshold
Threshold: 112
Clearing small objects took  17 seconds.
Feature finding and labeling took  22 seconds.
Number of wells: 16
At threshold
Threshold: 122
Clearing small objects took  17 seconds.
Feature finding and labeling took  21 seconds.
Number of wells: 16
At threshold
Threshold: 119
Clearing small objects took  16 seconds.
Feature finding and labeling took  20 seconds.
Number of wells: 16
At threshold
Threshold: 129
Clearing small objects took  17 seconds.
Feature finding and labeling took  22 seconds.
Number of wells: 16
At threshold
Threshold: 128
Clearing small objects took  16 seconds.
Feature finding and labeling took  20 seconds.
Number of wells: 16


,WellNo,centroid-0,centroid-1,bbox-0,bbox-1,bbox-2,bbox-3,area
0,1A,1528,7363,1079,5859,1978,8887,2208094
1,1B,2935,7376,2498,5859,3353,8878,2148123
2,1C,4389,7333,3933,5821,4825,8830,2220119
3,1D,5912,7359,5464,5832,6350,8845,2162821
4,2A,1342,2859,903,1325,1783,4374,2328732


In [39]:
test['Width']=test.apply(
    lambda row: row['bbox-3'] - row['bbox-1'], axis=1)
test.head()

,WellNo,centroid-0,centroid-1,bbox-0,bbox-1,bbox-2,bbox-3,area,Width
0,1A,1528,7363,1079,5859,1978,8887,2208094,3028
1,1B,2935,7376,2498,5859,3353,8878,2148123,3019
2,1C,4389,7333,3933,5821,4825,8830,2220119,3009
3,1D,5912,7359,5464,5832,6350,8845,2162821,3013
4,2A,1342,2859,903,1325,1783,4374,2328732,3049
